In [3]:
import psycopg2
import sklearn
from sklearn import svm
import pandas.io.sql as psql
import psycopg2
import re
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [4]:
import json
with open('/Users/krishna/MOOC/smart-city/login.json') as data_file:
    db = json.load(data_file)

conn = psycopg2.connect(database=db['dbname'], user=db['user'], password=db['password'], host=db['host'], port=db['port'])

In [5]:
dataframe = psql.frame_query("select idd, text, round(2*alch_score +3) as score from twitter.tweets where alch_score != 0 and alch_score is not null ;", conn)

/usr/local/lib/python2.7/site-packages/pandas/io/sql.py:1569: FutureWarning: frame_query is deprecated, use read_sql
  warnings.warn("frame_query is deprecated, use read_sql", FutureWarning)


In [23]:
dataframe = psql.frame_query("select idd, text, 2*alch_score +3 as alch_score_norm, local_score from twitter.tweets where (2*alch_score+3) - local_score < 1 and (2*alch_score+3 - local_score) > - 1 and alch_score != '0'  ;", conn)

In [24]:
dataframe.shape

(1471, 4)

In [25]:
def processTweet(tweet):
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+',' ',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet

In [26]:
dataframe['text']= dataframe['text'].apply(lambda x : processTweet(x))

In [36]:
dataframe['avg'] = dataframe.apply(lambda x : round((x['alch_score_norm'] + x['local_score'])/2) ,axis=1)
train_data, test_data = train_test_split(dataframe, test_size = 0.2)

In [37]:
train_data.head()

,idd,text,alch_score_norm,local_score,avg
1276,713501672979439616,rt bassem bravo il a reussi à réunir tt les qu...,3.717566,4,4
1366,713507521051422720,les arabes de lyon vont rien faire,3.682688,4,4
499,713319995464425472,rt [actu] location à lyon : « la persistance d...,1.554584,2,2
198,713432313686802432,"rt 🚨bassem vs booba🚨 : rdv le 25 mars au ""re...",1.554584,2,2
296,713183789506232320,le raté contre lyon ...... même à mes futurs p...,4.612864,4,4


In [38]:
vectorizer = TfidfVectorizer(min_df=5,
                             max_df = 1.0,
                             sublinear_tf=True,
                             use_idf=True)
train_vectors = vectorizer.fit_transform(train_data['text'])
test_vectors = vectorizer.transform(test_data['text'])

In [75]:
from sknn.mlp import Classifier, Layer
nn = Classifier(
    layers=[
        Layer("ExpLin", units=20),
        Layer("Softmax")],
    learning_rate=0.02,
    n_iter=30)

In [76]:
nn.fit(train_vectors, train_data['avg'])

Classifier(batch_size=1, callback=None, debug=False, dropout_rate=None,
      f_stable=0.001,
      hidden0=<sknn.nn.Layer `ExpLin`: units=20, name=u'hidden0', frozen=False>,
      layers=[<sknn.nn.Layer `ExpLin`: units=20, name=u'hidden0', frozen=False>, <sknn.nn.Layer `Softmax`: units=5, name=u'output', frozen=False>],
      learning_momentum=0.9, learning_rate=0.02, learning_rule=u'sgd',
      loss_type=None, n_iter=30, n_stable=10, normalize=None,
      output=<sknn.nn.Layer `Softmax`: units=5, name=u'output', frozen=False>,
      parameters=None, random_state=None, regularize=None, valid_set=None,
      valid_size=0.0, verbose=None, warning=None, weight_decay=None)

In [77]:
predict = nn.predict(test_vectors)
cm  = (confusion_matrix(test_data['avg'], predict))

[(295, 5)]


In [78]:
target_names = ['1', '2', '3','4','5']
print classification_report(test_data['avg'], predict, target_names=target_names)

             precision    recall  f1-score   support

          1       0.50      0.29      0.36         7
          2       0.73      0.81      0.77       120
          3       0.61      0.63      0.62        62
          4       0.82      0.74      0.78       103
          5       1.00      0.33      0.50         3

avg / total       0.73      0.73      0.73       295



In [79]:
cm

array([[ 2,  4,  1,  0,  0],
       [ 1, 97, 11, 11,  0],
       [ 0, 19, 39,  4,  0],
       [ 1, 13, 13, 76,  0],
       [ 0,  0,  0,  2,  1]])

In [80]:
print "Correctly Classified",cm.trace()
print "Total data point ",cm.sum()
print f1_score(test_data['avg'], predict, average="macro")

Correctly Classified 215
Total data point  295
0.604998521148
